# Análise de Dados Epidemiológicos: Dengue, Chikungunya, Zika e Febre Amarela

**Curso:** Bacharelado em Sistemas de Informação  
**Disciplina:** Ciência de Dados 1  
**Equipe:** Lucas André Oliveira Pinheiro, Eduardo Nogueira, Cynthia Neiva  
**Professor:**  Roberto Yuri da Silva Campo  
**Data:** 30/01/2026  


---
## ⚙️ Instalação de Dependências

Execute a célula abaixo **uma única vez** para instalar as bibliotecas necessárias.

In [58]:
# Execute esta célula uma vez e reinicie o kernel depois
import sys
!{sys.executable} -m pip install -q pandas numpy plotly scikit-learn requests openpyxl
print('Dependências instaladas! Reinicie o kernel se necessário (Kernel > Restart).')

Dependências instaladas! Reinicie o kernel se necessário (Kernel > Restart).


---
## 1️⃣ Introdução e Fundamentação Teórica

1️⃣ **Introdução e Fundamentação Teórica**

**1.1 Introdução à Ciência de Dados:**

A Ciência de Dados é uma área interdisciplinar que combina conhecimentos de
estatística, matemática, computação e conhecimento de domínio com o objetivo
de extrair informações relevantes e gerar valor a partir de grandes volumes
de dados estruturados e não estruturados.  

**1.2 Histórico e evolução da área:**  
A Ciência de Dados tem suas origens nas aplicações da estatística e da
computação para análise de dados ao longo do século XX. Inicialmente, essas
técnicas eram utilizadas de forma isolada em áreas como pesquisa científica,
economia e engenharia. Com o surgimento dos sistemas de bancos de dados e
posteriormente a evolução da mineiração de dados, tornou-se possível armazenar e analisar
volumes crescentes de informação de maneira estruturada.

No final dos anos 2000, a integração entre estatística, ciência da computação
e aprendizado de máquina passou a ser tratada de forma unificada, dando
origem à Ciência de Dados como um campo próprio. A partir desse período,
especialmente após 2008, a área consolidou-se tanto no meio acadêmico quanto
no mercado de trabalho, acompanhando a crescente digitalização da sociedade
e a intensificação da produção de dados.

**1.3 Importância da Ciência de Dados na sociedade atual:**  
Na sociedade contemporânea, a Ciência de Dados desempenha um papel fundamental
na tomada de decisões baseada em dados. Ela é amplamente utilizada em áreas
como saúde, finanças, educação, indústria, marketing e políticas públicas,
auxiliando na identificação de padrões, previsões de comportamentos e
otimização de processos, contribuindo para soluções mais eficientes e
inovadoras.

**1.4 Ciclo de vida da Ciência de Dados:**  
O ciclo de vida da Ciência de Dados é composto por etapas interdependentes,
que geralmente incluem:  
1. **Definição do problema:** Entendimento do objetivo e das perguntas a serem
   respondidas.

2. **Coleta de dados:** Obtenção dos dados a partir de diferentes fontes.

3. **Preparação e limpeza dos dados:** Tratamento de inconsistências, valores
   ausentes e padronização.

4. **Análise exploratória:** Compreensão dos dados por meio de estatísticas e
   visualizações.

5. **Modelagem:** Aplicação de técnicas estatísticas ou de aprendizado de máquina.

6. **Validação:** Verificação do desempenho e da qualidade dos resultados.

7. **Comunicação dos resultados:** Apresentação dos insights obtidos de forma
   clara e compreensível e utilização de gráficos interativos.




## 2️⃣ Coleta e Entendimento dos Dados

### 2.1 Fonte de Dados
Os dados foram obtidos da plataforma **DataSUS**, do Ministério da Saúde, via **SINAN** (Sistema de Informação de Agravos de Notificação). Para a Febre Amarela, os dados são monitorados pela **CGARB** (Coordenação-Geral de Vigilância de Arboviroses). A análise da Febre Amarela é restrita a **casos humanos** para manter comparabilidade metodológica.

### 2.2 Descrição dos Conjuntos de Dados
Os conjuntos apresentam variáveis:
- **Numéricas:** idade, ano, semana epidemiológica
- **Categóricas:** sexo, raça/cor, escolaridade, UF, classificação do caso
- **Temporais:** datas de início dos sintomas, notificação e óbito

### 2.3 Estratégia de Coleta
Os dados são baixados diretamente das URLs públicas do DataSUS em formato `.csv.zip` e `.json.zip`. Para otimizar memória, são selecionadas apenas as colunas relevantes à análise.

---
## ⚙️ Configuração do Ambiente

In [59]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import json
import warnings

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

# Paleta de cores padrão do projeto
CORES = {
    'Dengue':        '#E63946',
    'Chikungunya':   '#457B9D',
    'Zika':          '#2A9D8F',
    'Febre Amarela': '#E9A820'
}

print('Ambiente configurado com sucesso!')

Ambiente configurado com sucesso!


---
## 3️⃣ Importação e Manipulação dos Dados

In [60]:

# Funções auxiliares de download

0
def baixar_csv_zip(url, colunas=None):
    """Baixa um .csv.zip e retorna DataFrame com colunas selecionadas."""
    try:
        resp = requests.get(url, timeout=60)
        resp.raise_for_status()
        with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
            nome_csv = [n for n in z.namelist() if n.endswith('.csv')][0]
            with z.open(nome_csv) as f:
                df = pd.read_csv(f, sep=',', encoding='latin1',
                                 usecols=colunas, low_memory=True)
        return df
    except Exception as e:
        print(f'  Falha: {url.split("/")[-1]} — {e}')
        return pd.DataFrame()


def baixar_json_zip(url):
    """Baixa um .json.zip e retorna o conteúdo parseado."""
    resp = requests.get(url, timeout=60)
    resp.raise_for_status()
    with zipfile.ZipFile(io.BytesIO(resp.content)) as z:
        nome_json = [n for n in z.namelist() if n.endswith('.json')][0]
        with z.open(nome_json) as f:
            return json.load(f)

print('Funções auxiliares prontas.')

Funções auxiliares prontas.


In [61]:
# 3.1 Dengue — download 2015–2024

BASE_DENGUE = 'https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Dengue/csv/DENGBR{}.csv.zip'

COLUNAS_DENGUE = [
    'NU_ANO', 'SG_UF', 'CS_SEXO', 'NU_IDADE_N',
    'CS_RACA', 'CLASSI_FIN', 'EVOLUCAO', 'TPAUTOCTO', 'HOSPITALIZ'
]

dfs_dengue = []
for ano in range(15, 25):  # 2015–2024
    url = BASE_DENGUE.format(str(ano).zfill(2))
    print(f'  Dengue 20{ano:02d}...', end=' ')
    df = baixar_csv_zip(url, COLUNAS_DENGUE)
    if not df.empty:
        dfs_dengue.append(df)
        print(f'{len(df):,} registros')

dengue = pd.concat(dfs_dengue, ignore_index=True)
print(f'\n Dengue: {len(dengue):,} registros carregados')

  Dengue 2015... 2,398,060 registros
  Dengue 2016... 2,298,020 registros
  Dengue 2017... 518,483 registros
  Dengue 2018... 478,880 registros
  Dengue 2019... 2,261,956 registros
  Dengue 2020... 1,495,117 registros
  Dengue 2021... 1,010,359 registros
  Dengue 2022... 1,393,877 registros
  Dengue 2023... 1,645,350 registros
  Dengue 2024... 6,575,107 registros

 Dengue: 20,075,209 registros carregados


In [62]:
# 3.2 Chikungunya — download 2015–2024


BASE_CHIK = 'https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Chikungunya/csv/CHIKBR{}.csv.zip'

COLUNAS_CHIK = [
    'NU_ANO', 'SG_UF', 'CS_SEXO', 'NU_IDADE_N',
    'CS_RACA', 'CLASSI_FIN', 'EVOLUCAO', 'HOSPITALIZ'
]

dfs_chik = []
for ano in range(15, 25):
    url = BASE_CHIK.format(str(ano).zfill(2))
    print(f'  📥 Chikungunya 20{ano:02d}...', end=' ')
    df = baixar_csv_zip(url, COLUNAS_CHIK)
    if not df.empty:
        dfs_chik.append(df)
        print(f'{len(df):,} registros')

chikungunya = pd.concat(dfs_chik, ignore_index=True)
print(f'\n Chikungunya: {len(chikungunya):,} registros carregados')

  📥 Chikungunya 2015...   Falha: CHIKBR15.csv.zip — Usecols do not match columns, columns expected but not found: ['HOSPITALIZ']
  📥 Chikungunya 2016...   Falha: CHIKBR16.csv.zip — Usecols do not match columns, columns expected but not found: ['HOSPITALIZ']
  📥 Chikungunya 2017... 247,662 registros
  📥 Chikungunya 2018... 118,779 registros
  📥 Chikungunya 2019... 178,147 registros
  📥 Chikungunya 2020... 102,496 registros
  📥 Chikungunya 2021... 128,991 registros
  📥 Chikungunya 2022... 273,874 registros
  📥 Chikungunya 2023... 266,589 registros
  📥 Chikungunya 2024... 426,938 registros

 Chikungunya: 1,743,476 registros carregados


In [63]:

# 3.3 Zika — download 2016–2024


BASE_ZIKA = 'https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Zikavirus/csv/ZIKABR{}.csv.zip'

COLUNAS_ZIKA = [
    'NU_ANO', 'SG_UF', 'CS_SEXO', 'NU_IDADE_N',
    'CS_RACA', 'CLASSI_FIN', 'EVOLUCAO'
]

dfs_zika = []
for ano in range(16, 25):
    url = BASE_ZIKA.format(str(ano).zfill(2))
    print(f'   Zika 20{ano:02d}...', end=' ')
    df = baixar_csv_zip(url, COLUNAS_ZIKA)
    if not df.empty:
        dfs_zika.append(df)
        print(f'{len(df):,} registros')

zika = pd.concat(dfs_zika, ignore_index=True)
print(f'\n Zika: {len(zika):,} registros carregados')

   Zika 2016... 281,464 registros
   Zika 2017... 32,684 registros
   Zika 2018... 20,584 registros
   Zika 2019... 30,500 registros
   Zika 2020... 20,867 registros
   Zika 2021... 19,090 registros
   Zika 2022... 35,121 registros
   Zika 2023... 36,003 registros
   Zika 2024... 43,676 registros

 Zika: 519,989 registros carregados


In [64]:
# 3.4 Febre Amarela — download JSON (casos humanos)

URL_FA = 'https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/Febre+Amarela/json/fa_casoshumanos_1994-2025.json.zip'

print(' Febre Amarela (JSON)...')
dados_fa = baixar_json_zip(URL_FA)

if isinstance(dados_fa, list):
    febre_amarela = pd.json_normalize(dados_fa)
elif isinstance(dados_fa, dict):
    chave = list(dados_fa.keys())[0]
    febre_amarela = pd.json_normalize(dados_fa[chave])

# Filtrar somente casos humanos
if 'tipo_caso' in febre_amarela.columns:
    febre_amarela = febre_amarela[
        febre_amarela['tipo_caso'].str.upper().str.contains('HUMANO', na=False)
    ]

print(f' Febre Amarela: {len(febre_amarela):,} registros | Colunas: {list(febre_amarela.columns)}')
febre_amarela.head(3)

 Febre Amarela (JSON)...
 Febre Amarela: 2,886 registros | Colunas: ['COD_MUN_LPI', 'MACRORREG_LPI', 'IDADE', 'UF_LPI', 'SE_IS', 'OBITO', 'MONITORAMENTO_IS', 'MUN_LPI', 'SEXO', 'DT_OBITO', 'MES_IS', 'COD_UF_LPI', 'ID', 'DT_IS', 'ANO_IS']


,COD_MUN_LPI,MACRORREG_LPI,IDADE,UF_LPI,SE_IS,OBITO,MONITORAMENTO_IS,MUN_LPI,SEXO,DT_OBITO,MES_IS,COD_UF_LPI,ID,DT_IS,ANO_IS
0,140005.0,N,NaN,RR,48.0,SIM,1994/1995,ALTO ALEGRE,M,01/12/1994,11.0,14,1,29/11/1994,1994
1,140045.0,N,19.0,RR,8.0,NÃO,1994/1995,PACARAIMA,M,None,2.0,14,2,19/02/1995,1995
2,210060.0,NE,32.0,MA,13.0,IGN,1994/1995,AMARANTE DO MARANHÃO,M,None,4.0,21,3,01/04/1995,1995


In [65]:
# 3.5 Exploração inicial


for nome, df in [('Dengue', dengue), ('Chikungunya', chikungunya),
                 ('Zika', zika), ('Febre Amarela', febre_amarela)]:
    print(f'\n {nome}: {df.shape[0]:,} linhas × {df.shape[1]} colunas')
    display(df.head(2))


 Dengue: 20,075,209 linhas × 9 colunas


,NU_ANO,NU_IDADE_N,CS_SEXO,CS_RACA,SG_UF,HOSPITALIZ,TPAUTOCTO,CLASSI_FIN,EVOLUCAO
0,2015,4038.0,M,4.0,11.0,2.0,NaN,5.0,NaN
1,2015,4026.0,F,4.0,14.0,NaN,NaN,8.0,NaN



 Chikungunya: 1,743,476 linhas × 8 colunas


,NU_ANO,NU_IDADE_N,CS_SEXO,CS_RACA,SG_UF,HOSPITALIZ,CLASSI_FIN,EVOLUCAO
0,2017,4019.0,F,1.0,12,NaN,5.0,NaN
1,2017,4030.0,M,2.0,12,2.0,5.0,1.0



 Zika: 519,989 linhas × 7 colunas


,NU_ANO,NU_IDADE_N,CS_SEXO,CS_RACA,SG_UF,CLASSI_FIN,EVOLUCAO
0,2016,4116,M,NaN,17.0,8.0,NaN
1,2016,4048,M,4.0,29.0,8.0,1.0



 Febre Amarela: 2,886 linhas × 15 colunas


,COD_MUN_LPI,MACRORREG_LPI,IDADE,UF_LPI,SE_IS,OBITO,MONITORAMENTO_IS,MUN_LPI,SEXO,DT_OBITO,MES_IS,COD_UF_LPI,ID,DT_IS,ANO_IS
0,140005.0,N,NaN,RR,48.0,SIM,1994/1995,ALTO ALEGRE,M,01/12/1994,11.0,14,1,29/11/1994,1994
1,140045.0,N,19.0,RR,8.0,NÃO,1994/1995,PACARAIMA,M,None,2.0,14,2,19/02/1995,1995


---
## 4️⃣ Transformação e Integração dos Dados

In [ ]:
# 4.1 Mapeamentos padrão dados do SINAN


MAPA_RACA = {
    1: 'Branca', 2: 'Preta', 3: 'Amarela',
    4: 'Parda', 5: 'Indígena', 9: 'Ignorado'
}

MAPA_UF = {
    35: 'SP', 31: 'MG', 52: 'GO', 41: 'PR', 26: 'PE',
    23: 'CE', 29: 'BA', 50: 'MS', 33: 'RJ', 53: 'DF',
    24: 'RN', 51: 'MT', 32: 'ES', 42: 'SC', 25: 'PB',
    27: 'AL', 17: 'TO', 43: 'RS', 12: 'AC', 15: 'PA',
    22: 'PI', 13: 'AM', 28: 'SE', 14: 'RR', 16: 'AP',
    11: 'RO', 21: 'MA'
}

MAPA_EVOLUCAO = {
    1: 'Alta', 2: 'Óbito por doença',
    3: 'Óbito por outra causa', 9: 'Ignorado'
}

MAPA_CLASSI = {5: 'Confirmado', 8: 'Descartado', 10: 'Inconclusivo'}


def transformar_sinan(df, doenca):
    """Aplica mapeamentos e cria variáveis derivadas em datasets SINAN."""
    df = df.copy()

    # Ano
    df['NU_ANO'] = pd.to_numeric(df['NU_ANO'], errors='coerce').astype('Int16')

    # Idade em anos (código SINAN: unidade*1000 + valor)
    df['NU_IDADE_N'] = pd.to_numeric(df['NU_IDADE_N'], errors='coerce')
    unidade = df['NU_IDADE_N'] // 1000
    valor   = df['NU_IDADE_N'] % 1000
    df['IDADE_ANOS'] = np.where(unidade == 4, valor, np.nan).astype('float32')

    # Faixa etária
    bins   = [0, 4, 14, 24, 39, 59, 120]
    labels = ['0–4', '5–14', '15–24', '25–39', '40–59', '60+']
    df['FAIXA_ETARIA'] = pd.cut(df['IDADE_ANOS'], bins=bins, labels=labels, right=True)

    # Raça
    df['CS_RACA'] = pd.to_numeric(df['CS_RACA'], errors='coerce').map(MAPA_RACA)

    # UF
    df['SG_UF'] = pd.to_numeric(df['SG_UF'], errors='coerce').map(MAPA_UF)

    # Evolução e classificação
    if 'EVOLUCAO' in df.columns:
        df['EVOLUCAO'] = pd.to_numeric(df['EVOLUCAO'], errors='coerce').map(MAPA_EVOLUCAO)
    if 'CLASSI_FIN' in df.columns:
        df['CLASSI_FIN'] = pd.to_numeric(df['CLASSI_FIN'], errors='coerce').map(MAPA_CLASSI)

    # Período epidemiológico
    df['PERIODO'] = np.where(
        df['NU_ANO'] < 2020,
        'Pré-Pandemia (≤2019)',
        'Pandemia/Pós (2020+)'
    )

    df['DOENCA'] = doenca
    return df


dengue      = transformar_sinan(dengue,      'Dengue')
chikungunya = transformar_sinan(chikungunya, 'Chikungunya')
zika        = transformar_sinan(zika,        'Zika')

print(' Transformações aplicadas.')

In [ ]:

# 4.2 Transformação da Febre Amarela
# Obs: A fonte dela como foi especificado anteriormente é diferente do sinan e segue uma estrutura diferente, por isso precisou ser feita separadamente.


fa = febre_amarela.copy()
fa.columns = fa.columns.str.upper().str.strip()

# Detectar coluna de ano automaticamente
col_ano  = next((c for c in fa.columns if 'ANO' in c), None)
col_sexo = next((c for c in fa.columns if 'SEXO' in c), None)
col_uf   = next((c for c in fa.columns if 'UF' in c or 'ESTADO' in c), None)

renomear = {}
if col_ano:  renomear[col_ano]  = 'NU_ANO'
if col_sexo: renomear[col_sexo] = 'CS_SEXO'
if col_uf:   renomear[col_uf]   = 'SG_UF'
fa = fa.rename(columns=renomear)

if 'NU_ANO' in fa.columns:
    fa['NU_ANO'] = pd.to_numeric(fa['NU_ANO'], errors='coerce').astype('Int16')
    fa['PERIODO'] = np.where(fa['NU_ANO'] < 2020, 'Pré-Pandemia (≤2019)', 'Pandemia/Pós (2020+)')

fa['DOENCA'] = 'Febre Amarela'
print(f' Febre Amarela transformada: {len(fa):,} casos')

In [ ]:
# 4.3 Integração — série temporal unificada (JOIN por concat)


def agg_casos_ano(df, doenca):
    return (
        df.groupby('NU_ANO', observed=True)
          .size()
          .reset_index(name='CASOS')
          .assign(DOENCA=doenca)
    )

serie_temporal = pd.concat([
    agg_casos_ano(dengue,      'Dengue'),
    agg_casos_ano(chikungunya, 'Chikungunya'),
    agg_casos_ano(zika,        'Zika'),
    agg_casos_ano(fa,          'Febre Amarela'),
], ignore_index=True)

serie_temporal = serie_temporal.dropna(subset=['NU_ANO'])
serie_temporal['NU_ANO'] = serie_temporal['NU_ANO'].astype(int)

print(' Série temporal integrada (pivot):')
display(
    serie_temporal
    .pivot_table(index='NU_ANO', columns='DOENCA', values='CASOS', aggfunc='sum')
    .fillna(0).astype(int)
)

In [ ]:
# 4.4 Variáveis derivadas


# Taxa de hospitalização (Dengue e Chikungunya)
print('Taxa de hospitalização:')
for nome, df in [('Dengue', dengue), ('Chikungunya', chikungunya)]:
    if 'HOSPITALIZ' in df.columns:
        total = len(df)
        hosp  = (pd.to_numeric(df['HOSPITALIZ'], errors='coerce') == 1).sum()
        print(f'  {nome}: {hosp:,} hospitalizações / {total:,} casos ({hosp/total*100:.2f}%)')

# Média e desvio anual
print('\nMédia de casos anuais por doença:')
print(serie_temporal.groupby('DOENCA')['CASOS'].mean().round(0).astype(int))

---
## 5️⃣ Tratamento de Dados

In [ ]:
# 5.1 Diagnóstico de valores faltantes


def diagnostico_nulos(df, nome):
    nulos = df.isnull().sum()
    pct   = (nulos / len(df) * 100).round(2)
    resultado = pd.DataFrame({'Faltantes': nulos, '%': pct})
    resultado = resultado[resultado['Faltantes'] > 0].sort_values('%', ascending=False)
    print(f'\n {nome}:')
    display(resultado)

for nome, df in [('Dengue', dengue), ('Chikungunya', chikungunya), ('Zika', zika)]:
    diagnostico_nulos(df, nome)

In [ ]:
# 5.2 Tratamento de valores faltantes


def tratar_nulos(df):
    df = df.copy()
    # Sexo: ignorado → 'I'
    if 'CS_SEXO' in df.columns:
        df['CS_SEXO'] = df['CS_SEXO'].fillna('I')
    # Categóricas: NaN → 'Não informado'
    for col in ['CS_RACA', 'CLASSI_FIN', 'EVOLUCAO']:
        if col in df.columns:
            df[col] = df[col].fillna('Não informado')
    # Idade: mantida como NaN (impossível imputar)
    return df

dengue      = tratar_nulos(dengue)
chikungunya = tratar_nulos(chikungunya)
zika        = tratar_nulos(zika)

print(' Nulos tratados.')

In [ ]:
# 5.3 Outliers — idades biologicamente impossíveis


for df in [dengue, chikungunya, zika]:
    antes = df['IDADE_ANOS'].notna().sum()
    df.loc[(df['IDADE_ANOS'] < 0) | (df['IDADE_ANOS'] > 120), 'IDADE_ANOS'] = np.nan
    depois = df['IDADE_ANOS'].notna().sum()
    removidos = antes - depois
    if removidos > 0:
        print(f'  {df["DOENCA"].iloc[0]}: {removidos:,} outliers de idade removidos')

print(' Outliers tratados.')

In [ ]:
# 5.4 Normalização Z-score (para comparação entre doenças)


from sklearn.preprocessing import StandardScaler

pivot = (
    serie_temporal
    .pivot_table(index='NU_ANO', columns='DOENCA', values='CASOS', aggfunc='sum')
    .fillna(0)
)

scaler     = StandardScaler()
pivot_norm = pd.DataFrame(
    scaler.fit_transform(pivot),
    index=pivot.index,
    columns=pivot.columns
)

print(' Normalização Z-score aplicada:')
display(pivot_norm.round(2))

In [ ]:
# 5.5 Codificação de variáveis categóricas (Label Encoding)


from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
dengue['CS_SEXO_COD'] = le.fit_transform(dengue['CS_SEXO'].astype(str))

print('Codificação de CS_SEXO (Dengue):')
print(dict(zip(le.classes_, le.transform(le.classes_))))

---
## 6️⃣ Visualização Interativa de Dados

> Todos os gráficos abaixo são **interativos**: passe o mouse para ver valores, clique na legenda para filtrar doenças, use o zoom e os botões no canto superior direito.

In [ ]:

# 6.1 Gráfico de Linhas — Evolução temporal das arboviroses


fig = px.line(
    serie_temporal.sort_values('NU_ANO'),
    x='NU_ANO', y='CASOS', color='DOENCA',
    color_discrete_map=CORES,
    markers=True,
    title='Evolução Temporal das Arboviroses no Brasil',
    labels={'NU_ANO': 'Ano', 'CASOS': 'Casos Notificados', 'DOENCA': 'Doença'},
    hover_data={'CASOS': ':,'},
)

# Linha vertical indicando início da pandemia
fig.add_vline(
    x=2020, line_dash='dash', line_color='gray', opacity=0.6,
    annotation_text='Pandemia COVID-19', annotation_position='top right'
)

fig.update_layout(
    hovermode='x unified',
    xaxis=dict(dtick=1),
    yaxis_tickformat=',',
    legend_title='Doença',
    height=480,
    template='plotly_white'
)

fig.show()

In [ ]:
# 6.2 Gráfico de Linhas com escala logarítmica
# Obs: Peermite visualizar Zika e Febre Amarela junto com Dengue


fig_log = px.line(
    serie_temporal.sort_values('NU_ANO'),
    x='NU_ANO', y='CASOS', color='DOENCA',
    color_discrete_map=CORES,
    markers=True,
    log_y=True,
    title='Evolução Temporal das Arboviroses — Escala Logarítmica',
    labels={'NU_ANO': 'Ano', 'CASOS': 'Casos (escala log)', 'DOENCA': 'Doença'},
)

fig_log.add_vline(x=2020, line_dash='dash', line_color='gray', opacity=0.5)
fig_log.update_layout(
    hovermode='x unified', xaxis=dict(dtick=1),
    height=450, template='plotly_white'
)
fig_log.show()

In [ ]:
# 6.3 Gráfico de Barras — Top 15 UFs por casos de Dengue


uf_dengue = (
    dengue['SG_UF'].value_counts()
    .head(15)
    .reset_index()
    .rename(columns={'SG_UF': 'UF', 'count': 'Casos'})
    .sort_values('Casos', ascending=True)
)

fig_uf = px.bar(
    uf_dengue, x='Casos', y='UF',
    orientation='h',
    title='Dengue — 15 Estados com Maior Número de Casos (2015–2024)',
    labels={'Casos': 'Casos Notificados', 'UF': 'Estado'},
    color='Casos',
    color_continuous_scale='Reds',
    text_auto=','
)

fig_uf.update_layout(height=480, template='plotly_white', coloraxis_showscale=False)
fig_uf.update_traces(textposition='outside')
fig_uf.show()

In [ ]:

# 6.4 Histograma — Distribuição de Idade por Doença

# Montar tabela unificada com amostragem para performance
dfs_hist = []
for nome, df in [('Dengue', dengue), ('Chikungunya', chikungunya), ('Zika', zika)]:
    tmp = df[['IDADE_ANOS', 'DOENCA']].dropna()
    tmp = tmp[(tmp['IDADE_ANOS'] >= 0) & (tmp['IDADE_ANOS'] <= 100)]
    dfs_hist.append(tmp.sample(min(80_000, len(tmp)), random_state=42))

df_hist = pd.concat(dfs_hist)

fig_hist = px.histogram(
    df_hist, x='IDADE_ANOS', color='DOENCA',
    facet_col='DOENCA',
    nbins=40,
    color_discrete_map=CORES,
    title='Distribuição de Idade dos Casos por Doença',
    labels={'IDADE_ANOS': 'Idade (anos)', 'count': 'Frequência'},
    opacity=0.85,
)

fig_hist.update_layout(height=400, showlegend=False, template='plotly_white')
fig_hist.show()

In [52]:

# 6.5 Boxplot — Comparativo de Idade entre Doenças


fig_box = px.box(
    df_hist,
    x='DOENCA', y='IDADE_ANOS', color='DOENCA',
    color_discrete_map=CORES,
    title='Distribuição de Idade dos Casos — Dengue, Chikungunya e Zika',
    labels={'DOENCA': 'Doença', 'IDADE_ANOS': 'Idade (anos)'},
    points=False,
)

fig_box.update_layout(height=430, showlegend=False, template='plotly_white')
fig_box.show()

In [53]:
# 6.6 Gráfico de Barras Agrupadas — Casos por Sexo e Período


dfs_sexo = []
for nome, df in [('Dengue', dengue), ('Chikungunya', chikungunya), ('Zika', zika)]:
    tabela = (
        df[df['CS_SEXO'].isin(['F', 'M'])]
        .groupby(['PERIODO', 'CS_SEXO'], observed=True)
        .size()
        .reset_index(name='Casos')
        .assign(DOENCA=nome)
    )
    dfs_sexo.append(tabela)

df_sexo = pd.concat(dfs_sexo)
df_sexo['Sexo'] = df_sexo['CS_SEXO'].map({'F': 'Feminino', 'M': 'Masculino'})

fig_sexo = px.bar(
    df_sexo, x='PERIODO', y='Casos', color='Sexo',
    facet_col='DOENCA', barmode='group',
    color_discrete_sequence=['#E63946', '#457B9D'],
    title='Distribuição de Casos por Sexo e Período Epidemiológico',
    labels={'PERIODO': 'Período', 'Casos': 'Número de Casos'},
    text_auto=False,
)

fig_sexo.update_layout(height=430, template='plotly_white')
fig_sexo.update_xaxes(tickangle=10)
fig_sexo.show()

In [54]:
# 6.7 Heatmap — Intensidade epidemiológica normalizada (Z-score)


fig_heat = px.imshow(
    pivot_norm.T.round(2),
    color_continuous_scale='RdBu_r',
    color_continuous_midpoint=0,
    title='Intensidade Epidemiológica por Doença e Ano (Z-score)',
    labels={'x': 'Ano', 'y': 'Doença', 'color': 'Desvio (σ)'},
    aspect='auto',
    text_auto='.1f',
)

fig_heat.update_layout(height=320, template='plotly_white')
fig_heat.show()

In [ ]:
# 6.8 Gráfico de Barras — Distribuição por Raça/Cor (Dengue)


raca_df = (
    dengue[~dengue['CS_RACA'].isin(['Não informado', 'Ignorado'])]
    ['CS_RACA'].value_counts()
    .reset_index()
    .rename(columns={'CS_RACA': 'Raça/Cor', 'count': 'Casos'})
    .sort_values('Casos', ascending=True)
)

fig_raca = px.bar(
    raca_df, x='Casos', y='Raça/Cor',
    orientation='h',
    title='Dengue — Distribuição de Casos por Raça/Cor (2015–2024)',
    color='Casos', color_continuous_scale='Blues',
    text_auto=','
)

fig_raca.update_layout(height=380, template='plotly_white', coloraxis_showscale=False)
fig_raca.update_traces(textposition='outside')
fig_raca.show()

In [ ]:
# 6.9 Gráfico de Barras — Total de casos por arbovirose


totais = (
    serie_temporal.groupby('DOENCA')['CASOS']
    .sum()
    .reset_index()
    .sort_values('CASOS', ascending=False)
)

fig_tot = px.bar(
    totais, x='DOENCA', y='CASOS',
    color='DOENCA',
    color_discrete_map=CORES,
    title='Total de Casos Notificados por Arbovirose no Brasil',
    labels={'DOENCA': 'Doença', 'CASOS': 'Total de Casos'},
    text_auto=','
)

fig_tot.update_layout(height=420, showlegend=False, template='plotly_white')
fig_tot.update_traces(textposition='outside')
fig_tot.show()

In [57]:

# 6.10 Gráfico de Área Empilhada — Composição anual por doença

# Apenas Dengue, Chikungunya e Zika (mesmos períodos)
df_area = serie_temporal[
    serie_temporal['DOENCA'].isin(['Dengue', 'Chikungunya', 'Zika'])
].sort_values('NU_ANO')

fig_area = px.area(
    df_area, x='NU_ANO', y='CASOS', color='DOENCA',
    color_discrete_map=CORES,
    title='Composição Anual de Casos — Dengue, Chikungunya e Zika',
    labels={'NU_ANO': 'Ano', 'CASOS': 'Casos Notificados', 'DOENCA': 'Doença'},
    groupnorm='',   # empilhado absoluto
)

fig_area.update_layout(
    hovermode='x unified', xaxis=dict(dtick=1),
    height=430, template='plotly_white'
)
fig_area.show()

---
## 7️⃣ Produto Final — Resultados e Conclusões

### 7.1 Síntese dos Resultados

A análise epidemiológica das arboviroses no Brasil, baseada em dados do SINAN/DataSUS, revelou padrões relevantes:

**Dengue** é a arbovirose de maior impacto em volume absoluto de casos, com picos expressivos que refletem surtos epidêmicos cíclicos, típicos da dinâmica de imunidade populacional ao vírus.

**Chikungunya** emergiu com força a partir de 2015 e manteve circulação expressiva, especialmente em regiões de alta densidade vetorial.

**Zika** apresentou pico epidêmico em 2015–2016 seguido de queda abrupta,  um padrão consistente com a aquisição de imunidade coletiva após o surto inicial.

**Febre Amarela** apresenta volume de casos muito inferior, com surtos concentrados em anos específicos, refletindo a natureza focal e sazonal da doença.

### 7.2 Padrões Epidemiológicos Observados

- **Sazonalidade:** todas as arboviroses apresentam variação temporal, com maior incidência em períodos chuvosos (favorecendo a proliferação do *Aedes aegypti*).
- **Distribuição geográfica:** São Paulo, Minas Gerais e Goiás concentram a maioria dos casos de Dengue, refletindo densidade populacional e urbanização.
- **Perfil demográfico:** maior proporção de casos em adultos jovens; sexo feminino ligeiramente predominante, possivelmente por maior exposição domiciliar ao vetor.
- **Impacto da pandemia COVID-19:** possível subnotificação visível nas séries temporais a partir de 2020.

### 7.3 Implicações para a Saúde Pública

Os achados reforçam a necessidade de vigilância epidemiológica ativa, intensificação do controle vetorial em períodos de maior risco, monitoramento de grupos vulneráveis e uso de ciência de dados para antecipar surtos e direcionar recursos.

### 7.4 Limitações

- **Subnotificação:** casos leves raramente chegam ao sistema de saúde
- **Inconsistências nas bases:** dados faltantes e divergências entre anos
- **Ausência de variáveis externas:** fatores climáticos, socioeconômicos e de cobertura vacinal não foram integrados
- **Análise descritiva:** não foram realizadas modelagens preditivas

### 7.5 Conclusão

Este projeto demonstra como a Ciência de Dados pode ser aplicada de forma prática no apoio à análise epidemiológica, contribuindo para a compreensão da dinâmica das arboviroses no Brasil e oferecendo subsídios para reflexões sobre vigilância epidemiológica e políticas públicas de saúde.